# Object Identification (TensorFlow Hub) - Local Images

This notebook identifies whether a given object is present in a set of images. If the object is found, it returns a dataframe with the file name, score and bounding boxes.

We are using a dataset from [UCF](https://www.crcv.ucf.edu/data/GMCP_Geolocalization/#Dataset) that contains 62,058 high quality Google Street View images, and [TensorFlow Hub](https://www.tensorflow.org/hub), a library and platform for transfer learning.

## Set Up Environment

This notebook has been tested with the following package versions:

In [1]:
# Python 3.10.6
!pip install matplotlib==3.6.2
!pip install Pillow==9.3.0
!pip install tensorflow==2.10.0
!pip install tensorflow-hub==0.12.0
!pip install urllib3==1.26.12
!pip install verta==0.21.0

## Imports

In [ ]:
import numpy as np
import os
import pandas as pd
import tempfile
import tensorflow as tf
import tensorflow_hub as hub
import time
import urllib.request
import warnings

from PIL import Image, ImageOps
from verta import Client
from verta.environment import Python
from verta.registry import VertaModelBase, verify_io
from verta.utils import ModelAPI

warnings.filterwarnings('ignore', category = FutureWarning)

In [ ]:
print(f"Version: {tf.__version__}")
print(f"Eager mode: {tf.executing_eagerly()}")
print(f"Hub version: {hub.__version__}")
print(f'GPU is', 'AVAILABLE.' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE.')

## Set Up Verta

In [ ]:
PROJECT_NAME = 'Object-Detection'

os.environ['VERTA_HOST'] = 'app.verta.ai'
os.environ['VERTA_EMAIL'] = ''
os.environ['VERTA_DEV_KEY'] = ''

client = Client(os.environ['VERTA_HOST'], use_git = False)
proj = client.set_project(PROJECT_NAME)

## Define Model Class

In [ ]:
class DetectObject(VertaModelBase):
    def __init__(self, artifacts = None):
        module_handle = 'https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1'
        self.detector = hub.load(module_handle).signatures['default']
    
    def handle_img(self, img, width = 256, height = 256):
        _, path = tempfile.mkstemp(suffix = '.jpg')
        img = np.array(img, dtype = np.uint8)
        img = Image.fromarray(img)
        img = ImageOps.grayscale(img)
        img.thumbnail((width, height), Image.Resampling.LANCZOS)
        img.save(path, format = 'JPEG', quality = 90)
        
        print(f"Image downloaded to {path}.")
        return path

    def load_img(self, path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels = 3)
        
        return img

    def filter_results(self, file, response, entity = 'Car', min_score = .2):
        unused_keys = ['detection_class_labels', 'detection_class_names']
        response = {key: value.numpy().tolist() for key, value in response.items()}
        response = dict([(key, val) for key, val in response.items() if key not in unused_keys])
        response['detection_class_entities'] = [v.decode() for v in response['detection_class_entities']]

        entities = response['detection_class_entities']
        scores = response['detection_scores']
        bboxes = response['detection_boxes']
        result = {}

        for i in range(len(entities)):
            if entities[i] == entity and scores[i] >= min_score:
                ymin, xmin, ymax, xmax = bboxes[i]
                result = {
                    'file': file,
                    'has_car': 1,
                    'score': scores[i],
                    'bboxes': {'ymin': ymin, 'xmin': xmin, 'ymax': ymax, 'xmax': xmax}
                }
                break

        if len(result) == 0:
            result = {
                'file': file,
                'has_car': 0,
                'score': 0,
                'bboxes': {'ymin': 0, 'xmin': 0, 'ymax': 0, 'xmax': 0}
            }
        
        print(f"Found {result['has_car']} object(s).")
        return result

    def run_detector(self, file, image_path):
        img = self.load_img(image_path)
        img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
        
        response = self.detector(img)
        result = self.filter_results(file, response)
        
        return result

    def detect_objects(self, file, img_arr):
        start_time = time.time()
        image_path = self.handle_img(img_arr, 640, 480)
        result = self.run_detector(file, image_path)
        end_time = time.time()

        print(f"Inference time: {end_time - start_time}.")
        return result

    @verify_io
    def predict(self, data):
        result = self.detect_objects(data[0], data[1])

        return result

## Register the Model

In [ ]:
registered_model = client.get_or_create_registered_model(
    name = 'object-detection', labels = ['object-detection']
)

In [2]:
input = {
    "file_name": "",
    "image_arr": []
}

In [3]:
output = {
    "file_name": "",
    "has_car": 0,
    "score": 0,
    "ymin": 0,
    "xmin": 0,
    "ymax": 0,
    "xmax": 0
}

In [4]:
model_version = registered_model.create_standard_model(
    model_cls = DetectObject,
    environment = Python(requirements = ['tensorflow', 'tensorflow_hub', 'matplotlib']),
    model_api = ModelAPI([input], [output]),
    name = 'v1'
)

## Deploy Model to Endpoint

In [ ]:
endpoint = client.get_or_create_endpoint('object-detection')

In [ ]:
endpoint.update(model_version, wait = True)

In [ ]:
deployed_model = endpoint.get_deployed_model()

## Download Images

In [2]:
!mkdir images

In [ ]:
urls = [
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_0.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_1.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_2.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_3.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_4.jpg',
    'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/images/000001_5.jpg'
]

for url in urls:
    urllib.request.urlretrieve(url, f"images/{url.split('/')[-1]}")

## Run

In [ ]:
folder = 'images/'
results = []

for root, dirs, files in os.walk(folder):    
    for file in sorted(files):
        print(f"Processing {file}...")
        img = Image.open(f"{root}/{file}")
        img_arr = np.array(img).tolist()
        result = deployed_model.predict([file, img_arr])
        results.append(result)
        break

## Present Results

In [ ]:
cols = list(results[0].keys())[:-1]
cols.extend(list(results[0]['bboxes'].keys()))
data = []

for item in results:
    values = list(item.values())[0:3]
    values.extend(list(item['bboxes'].values()))
    data.append(values)
    
df_output = pd.DataFrame(data, columns = cols)

In [ ]:
df_output